# 0. Install Dependencies

In [1]:
!pip install tensorflow==2.5.0
!pip install gym
!pip install keras
!pip install keras-rl2

You should consider upgrading via the 'C:\Users\user\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\user\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\user\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\user\anaconda3\python.exe -m pip install --upgrade pip' command.


# 1. Test Random Environment with OpenAI Gym

In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random


In [17]:
import numpy as np
import re as regex
import random
import math

# board is the visual presentation of the board, fboard is the flattened version for the ML
class Game:
    def __init__(self, size):
        self.board = []
        self.board_size = size

    def createboard(self):
        self.board = [['' for _ in range(self.board_size)] for _ in range(self.board_size)]
        self.board[random.randint(0, self.board_size - 1)][random.randint(0, self.board_size - 1)] = 2
        self.board[random.randint(0, self.board_size - 1)][random.randint(0, self.board_size - 1)] = 4

    def getboard(self) -> list:
        k = np.reshape(self.board, (1, -1))
        k[k == ''] = 0
#         k = np.log(k, 2)
        k = list(map(int, k[0]))
        return np.array(k)

    def printboard(self) -> None:
        for r in self.board:
            print(r, '\t')

    # push the array to the left
    def c_dir(self) -> None:
        for r, re in enumerate(self.board):
            # table test for unique values
            table = np.unique(re).tolist()
            table.remove('') if '' in table else ''
            # table = list(filter(('').__ne__, table))
            table = list(map(int, table))

            # turn the list into a text
            t = '_' + '__'.join(map(str, re)) + '_'

            # use regex replace method to combine unique values
            for v in table:
                t = regex.sub(f'_{v}_+{v}_', f'{int(v) * 2}._', t)

            # filter out the '' values
            l = regex.split('\_|\.', t)
            l = list(filter(('').__ne__, l))
            # set size to the board_size
            l += ['' for _ in range(self.board_size)]
            l = l[:self.board_size]

            self.board[r] = l

    # returns whether anything changed on the board
    def push_dir(self, dir=2) -> bool:
        # Apply action
        # 0 = up
        # 1 = down
        # 2 = left
        # 3 = right

        old_board = self.getboard()
        if dir == 2:
            self.c_dir()
        elif dir == 0:
            self.board = np.transpose(self.board).tolist()
            self.c_dir()
            self.board = np.transpose(self.board).tolist()
        elif dir == 3:
            self.board = np.fliplr(self.board).tolist()
            self.c_dir()
            self.board = np.fliplr(self.board).tolist()
        elif dir == 1:
            self.board = np.fliplr(np.transpose(self.board)).tolist()
            self.c_dir()
            self.board = np.transpose(np.fliplr(self.board)).tolist()

        comparison = old_board == self.getboard()
        if comparison.all():
            return False
        else:
            return True

    # place random
    def place_random(self):
        pool = []
        for r in range(self.board_size):
            for c in range(self.board_size):
                if self.board[r][c] == '':
                    pool.append((r, c))

        if len(pool) == 0:
            return

        r, c = random.choice(pool)
        self.board[r][c] = random.choice(['2', '2', '4'])

    # check if the AI lost - no more adjacent tiles with same value
    def check_game_ends(self) -> bool:
        for r in range(self.board_size):
            for c in range(self.board_size):
                if self.board[r][c] == '':
                    return False

                if r < self.board_size - 1:
                    if self.board[r][c] == self.board[r + 1][c]:
                        return False

                if c < self.board_size - 1:
                    if self.board[r][c] == self.board[r][c + 1]:
                        return False
                continue
        return True

    def current_v(self) -> int:
        counts = np.count_nonzero(self.getboard()==0)
        return counts

In [18]:
class GameEnv(Env):
    def __init__(self):
        # Actions we can take, up down left right
        self.action_space = Discrete(4)
        # Temperature array
#         self.observation_space = Box(low=np.array([0]), high=np.array([20]))
        observation_high = np.array([
          np.finfo(np.float16).max,
          np.finfo(np.float16).max,
          np.finfo(np.float16).max,
          np.finfo(np.float16).max,
          np.finfo(np.float16).max,
          np.finfo(np.float16).max,
          np.finfo(np.float16).max,
          np.finfo(np.float16).max,
          np.finfo(np.float16).max,
          np.finfo(np.float16).max,
          np.finfo(np.float16).max,
          np.finfo(np.float16).max,
          np.finfo(np.float16).max,
          np.finfo(np.float16).max,
          np.finfo(np.float16).max,
          np.finfo(np.float16).max])
        
        self.observation_space = Box(-observation_high, observation_high)

#         self.observation_space = Box(low=0, high=np.inf, shape=(1,16))
        
        # Set start game
        self.game = Game(size=4)
        self.game.createboard()
        self.game.printboard()
        
    def step(self, action):
        
        old_v = self.game.current_v()
        
        # Apply action
        # 0 = up
        # 1 = down
        # 2 = left
        # 3 = right
        moved = self.game.push_dir(action)
        
        
        # Calculate reward
        reward = self.game.current_v() - old_v
        
        if not moved:
            reward = -1
        else:
            self.game.place_random()
        
#         print(f'rewards: {reward} {type(reward)}')
#         self.game.printboard()
#         print('')
        
        
        # Check if game is finished
        done = True if self.game.check_game_ends() else False
        if done:
            reward = -1
            
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        
        # Set placeholder for info
        info = {}
        
        # Return step information
        self.state = self.game.getboard()
        
        return self.state, reward, done, info

    def render(self):
        self.game.printboard()
        print('\n')
    
    def reset(self):
        # Reset shower temperature
        self.game = Game(size=4)
        self.game.createboard()
        
        return self.game.getboard()
    

In [19]:
env = GameEnv()

['', '', '', 2] 	
['', 4, '', ''] 	
['', '', '', ''] 	
['', '', '', ''] 	


In [20]:
env.observation_space.sample()

array([ 15864.045, -29152.457,  45612.32 , -50509.758, -30768.107,
       -48220.83 , -12865.95 ,  33512.402, -25808.098,  29717.51 ,
       -49796.723,  53998.22 , -13090.455,   5187.842, -19062.559,
       -50114.55 ], dtype=float32)

In [21]:
env.action_space.sample()

0

In [22]:
env.reset()

TypeError: return arrays must be of ArrayType

In [9]:
env.game.place_random()

print(env.game.board)
print(env.game.getboard())
print(env.game.current_v())
env.game.printboard()

[['', '', '', ''], [4, '', '', ''], ['2', '', '', 2], ['', '', '', '']]
[0 0 0 0 4 0 0 0 2 0 0 2 0 0 0 0]
13
['', '', '', ''] 	
[4, '', '', ''] 	
['2', '', '', 2] 	
['', '', '', ''] 	


In [10]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
#         print('action: ', action, type(action))
        
        n_state, reward, done, info = env.step(action)
        score+=reward
        
    print('Episode:{} Score:{}'.format(episode, score))

['', '', '', ''] 	
['', '', 2, ''] 	
['', '', 4, ''] 	
['', '', '', ''] 	


['', '', '', '4'] 	
['2', '', '', ''] 	
['4', '', '', ''] 	
['', '', '', ''] 	


['4', '', '', ''] 	
['2', '', '', ''] 	
['4', '', '', ''] 	
['', '', '', '2'] 	


['4', '', '', '2'] 	
['2', '', '', ''] 	
['4', '', '', ''] 	
['', '', '', '2'] 	


['4', '', '', '4'] 	
['2', '', '', ''] 	
['4', '2', '', ''] 	
['', '', '', ''] 	


['8', '', '', ''] 	
['2', '', '2', ''] 	
['4', '2', '', ''] 	
['', '', '', ''] 	


['8', '', '', ''] 	
['4', '', '', ''] 	
['4', '2', '4', ''] 	
['', '', '', ''] 	


['8', '', '', ''] 	
['4', '', '', ''] 	
['4', '2', '4', ''] 	
['', '', '', ''] 	


['', '', '', '8'] 	
['', '', '2', '4'] 	
['', '4', '2', '4'] 	
['', '', '', ''] 	


['', '', '', '8'] 	
['', '', '2', '4'] 	
['', '4', '2', '4'] 	
['', '', '', ''] 	


['', '4', '4', '8'] 	
['', '', '', '8'] 	
['', '2', '', ''] 	
['', '', '', ''] 	


['', '', '', ''] 	
['', '', '', ''] 	
['', '4', '2', ''] 	
['', '2', '4', '16'] 	


['', '4', '

['8', '8', '', ''] 	
['8', '2', '4', ''] 	
['2', '8', '16', '4'] 	


['', '', '2', '16'] 	
['', '4', '', '16'] 	
['', '8', '2', '4'] 	
['2', '8', '16', '4'] 	


['2', '16', '2', ''] 	
['4', '16', '', ''] 	
['8', '2', '4', ''] 	
['2', '8', '16', '4'] 	


['2', '', '2', ''] 	
['4', '32', '2', ''] 	
['8', '2', '4', ''] 	
['2', '8', '16', '4'] 	


['2', '32', '4', '4'] 	
['4', '2', '4', ''] 	
['8', '8', '16', ''] 	
['2', '2', '', ''] 	


['2', '32', '8', '4'] 	
['4', '2', '16', ''] 	
['8', '8', '', ''] 	
['2', '2', '4', ''] 	


['2', '32', '8', '4'] 	
['4', '2', '16', ''] 	
['16', '', '', ''] 	
['4', '4', '2', ''] 	


['2', '32', '8', '4'] 	
['4', '2', '16', ''] 	
['16', '', '', ''] 	
['8', '2', '2', ''] 	


['2', '', '4', ''] 	
['4', '', '8', ''] 	
['16', '32', '16', ''] 	
['8', '4', '2', '4'] 	


['2', '32', '4', '4'] 	
['4', '4', '8', ''] 	
['16', '', '16', ''] 	
['8', '2', '2', ''] 	


['', '2', '32', '8'] 	
['', '', '8', '8'] 	
['', '', '', '32'] 	
['', '4', '8', '4'] 	


['', '', '',


['2', '', '', '2'] 	
['4', '8', '', ''] 	
['2', '16', '', '2'] 	
['4', '2', '8', '4'] 	


['2', '8', '8', '4'] 	
['4', '16', '', '4'] 	
['2', '2', '2', ''] 	
['4', '', '', ''] 	


['2', '16', '4', ''] 	
['4', '16', '4', ''] 	
['4', '2', '', ''] 	
['4', '', '2', ''] 	


['2', '16', '4', ''] 	
['4', '16', '4', ''] 	
['4', '2', '', ''] 	
['4', '2', '2', ''] 	


['', '', '4', ''] 	
['2', '', '', ''] 	
['4', '32', '8', ''] 	
['8', '4', '2', ''] 	


['', '', '', '4'] 	
['', '', '2', '2'] 	
['', '4', '32', '8'] 	
['', '8', '4', '2'] 	


['', '4', '2', '4'] 	
['', '8', '32', '2'] 	
['', '', '4', '8'] 	
['4', '', '', '2'] 	


['4', '2', '4', ''] 	
['8', '32', '2', ''] 	
['4', '8', '', '2'] 	
['4', '2', '', ''] 	


['4', '2', '4', ''] 	
['8', '32', '2', ''] 	
['4', '8', '2', ''] 	
['4', '2', '2', ''] 	


['4', '2', '4', ''] 	
['8', '32', '2', ''] 	
['4', '8', '2', ''] 	
['4', '4', '', '4'] 	


['4', '2', '4', '4'] 	
['8', '32', '4', ''] 	
['8', '8', '', ''] 	
['', '4', '', '4'] 	


['', '2', ''

['2', '64', '4', '16'] 	


['', '', '4', '8'] 	
['', '2', '128', '2'] 	
['4', '32', '16', '8'] 	
['2', '64', '4', '16'] 	


['', '', '4', '8'] 	
['', '2', '128', '2'] 	
['4', '32', '16', '8'] 	
['2', '64', '4', '16'] 	


['4', '8', '', '2'] 	
['2', '128', '2', ''] 	
['4', '32', '16', '8'] 	
['2', '64', '4', '16'] 	


['4', '8', '2', '4'] 	
['2', '128', '2', ''] 	
['4', '32', '16', '8'] 	
['2', '64', '4', '16'] 	


['4', '8', '4', '4'] 	
['2', '128', '16', '8'] 	
['4', '32', '4', '16'] 	
['2', '64', '', '4'] 	


['2', '4', '8', '8'] 	
['2', '128', '16', '8'] 	
['4', '32', '4', '16'] 	
['', '2', '64', '4'] 	


['2', '2', '4', '16'] 	
['2', '128', '16', '8'] 	
['4', '32', '4', '16'] 	
['', '2', '64', '4'] 	


['4', '4', '16', ''] 	
['2', '128', '16', '8'] 	
['4', '32', '4', '16'] 	
['2', '64', '4', '2'] 	


['4', '4', '32', '8'] 	
['2', '128', '8', '16'] 	
['4', '32', '4', '2'] 	
['2', '64', '', ''] 	


['4', '4', '32', '8'] 	
['2', '128', '8', '16'] 	
['4', '32', '4', '2'] 	
['2', '64', 

['', '', '', '4'] 	
['', '', '2', '4'] 	
['', '', '4', '16'] 	
['2', '8', '2', '4'] 	


['4', '', '', ''] 	
['2', '4', '', '2'] 	
['4', '16', '', ''] 	
['2', '8', '2', '4'] 	


['', '', '', '4'] 	
['2', '2', '4', '2'] 	
['', '', '4', '16'] 	
['2', '8', '2', '4'] 	


['2', '', '', '4'] 	
['', '', '', '2'] 	
['', '2', '8', '16'] 	
['4', '8', '2', '4'] 	


['', '2', '2', '4'] 	
['', '', '', '2'] 	
['', '2', '8', '16'] 	
['4', '8', '2', '4'] 	


['', '', '4', '4'] 	
['', '', '2', '2'] 	
['', '2', '8', '16'] 	
['4', '8', '2', '4'] 	


['8', '', '2', ''] 	
['4', '', '', ''] 	
['2', '8', '16', ''] 	
['4', '8', '2', '4'] 	


['8', '16', '2', '4'] 	
['4', '2', '16', ''] 	
['2', '', '2', ''] 	
['4', '', '', ''] 	


['8', '16', '2', '4'] 	
['4', '2', '16', ''] 	
['4', '', '', ''] 	
['4', '2', '', ''] 	


['8', '16', '2', '4'] 	
['4', '2', '16', ''] 	
['4', '', '', ''] 	
['4', '2', '', ''] 	


['8', '16', '2', '4'] 	
['4', '2', '16', ''] 	
['4', '', '', ''] 	
['4', '2', '', ''] 	


['8', '16', '2'

['', '', '2', '8'] 	
['2', '64', '4', '16'] 	
['8', '16', '64', '2'] 	
['16', '4', '8', '64'] 	


['2', '64', '2', '8'] 	
['8', '16', '4', '16'] 	
['16', '4', '64', '2'] 	
['', '2', '8', '64'] 	


['2', '64', '2', '8'] 	
['8', '16', '4', '16'] 	
['16', '4', '64', '2'] 	
['', '2', '8', '64'] 	


Episode:6 Score:75
['', '', 4, ''] 	
[2, '', '', ''] 	
['', '', '', ''] 	
['', '', '', ''] 	


['4', '', '', ''] 	
['2', '', '2', ''] 	
['', '', '', ''] 	
['', '', '', ''] 	


['4', '', '', ''] 	
['4', '', '', ''] 	
['2', '', '', ''] 	
['', '', '', ''] 	


['', '', '', ''] 	
['4', '', '', ''] 	
['8', '', '', ''] 	
['2', '', '', ''] 	


['4', '', '', ''] 	
['8', '', '', ''] 	
['2', '', '', ''] 	
['', '', '', '2'] 	


['4', '', '', ''] 	
['8', '4', '', ''] 	
['2', '', '', ''] 	
['2', '', '', ''] 	


['', '', '4', ''] 	
['4', '', '', ''] 	
['8', '', '', ''] 	
['4', '4', '', ''] 	


['4', '', '', ''] 	
['4', '', '', ''] 	
['8', '', '2', ''] 	
['8', '', '', ''] 	


['', '', '', ''] 	
['', '', '', '']

['16', '4', '', ''] 	
['', '', '', '4'] 	


['', '2', '', ''] 	
['4', '2', '', '4'] 	
['2', '8', '16', '2'] 	
['16', '4', '8', '4'] 	


['4', '4', '16', '4'] 	
['2', '8', '8', '2'] 	
['16', '4', '', '4'] 	
['', '', '2', ''] 	


['', '8', '16', '4'] 	
['', '2', '16', '2'] 	
['2', '', '16', '8'] 	
['', '', '', '2'] 	


['8', '16', '4', ''] 	
['2', '16', '2', ''] 	
['2', '16', '8', ''] 	
['2', '', '', '4'] 	


['8', '32', '4', '4'] 	
['4', '16', '2', ''] 	
['2', '', '8', '2'] 	
['', '', '', ''] 	


['', '8', '32', '8'] 	
['', '4', '16', '2'] 	
['', '2', '8', '2'] 	
['', '', '4', ''] 	


['', '8', '32', '8'] 	
['', '4', '16', '4'] 	
['', '2', '8', ''] 	
['', '2', '4', ''] 	


['', '4', '32', ''] 	
['', '8', '16', ''] 	
['', '4', '8', '8'] 	
['', '4', '4', '4'] 	


['', '4', '32', '8'] 	
['', '8', '16', '4'] 	
['2', '8', '8', ''] 	
['', '', '4', ''] 	


['4', '', '32', ''] 	
['', '', '16', ''] 	
['', '4', '8', '8'] 	
['2', '16', '4', '4'] 	


['', '', '4', '32'] 	
['', '', '', '16'] 	
['', 

['16', '4', '64', '4'] 	
['2', '64', '16', '2'] 	
['4', '16', '8', '64'] 	
['8', '', '4', ''] 	


['16', '2', '64', ''] 	
['2', '4', '16', '4'] 	
['4', '64', '8', '2'] 	
['8', '16', '4', '64'] 	


Episode:8 Score:73
['', '', '', ''] 	
['', '', '', ''] 	
[2, '', '', ''] 	
['', '', 4, ''] 	


['', '2', '', ''] 	
['', '', '', ''] 	
['2', '', '', ''] 	
['4', '', '', ''] 	


['', '', '', '2'] 	
['', '', '2', ''] 	
['', '', '', '2'] 	
['', '', '', '4'] 	


['', '', '2', '4'] 	
['', '', '4', '4'] 	
['', '', '', ''] 	
['', '', '', ''] 	


['', '', '2', '4'] 	
['', '', '', '8'] 	
['', '', '', '2'] 	
['', '', '', ''] 	


['2', '4', '', ''] 	
['8', '2', '', ''] 	
['2', '', '', ''] 	
['', '', '', ''] 	


['', '', '', ''] 	
['2', '', '', ''] 	
['8', '4', '4', ''] 	
['2', '2', '', ''] 	


['', '', '', ''] 	
['', '', '', '2'] 	
['2', '', '8', '8'] 	
['', '', '', '4'] 	


['', '', '2', ''] 	
['2', '', '', ''] 	
['2', '16', '', ''] 	
['4', '', '', ''] 	


['2', '', '', ''] 	
['2', '', '', ''] 	
['2', '



['4', '16', '2', ''] 	
['2', '16', '128', '4'] 	
['2', '8', '32', ''] 	
['4', '8', '2', '2'] 	


['', '4', '16', '2'] 	
['2', '16', '128', '4'] 	
['', '2', '8', '32'] 	
['2', '4', '8', '4'] 	


['', '4', '16', '2'] 	
['2', '16', '128', '4'] 	
['', '2', '8', '32'] 	
['2', '4', '8', '4'] 	


['4', '16', '2', '4'] 	
['2', '16', '128', '4'] 	
['2', '8', '32', ''] 	
['2', '4', '8', '4'] 	


['4', '32', '2', '8'] 	
['4', '8', '128', '4'] 	
['2', '4', '32', ''] 	
['2', '', '8', ''] 	


['', '', '2', '2'] 	
['', '32', '128', ''] 	
['8', '8', '32', '8'] 	
['4', '4', '8', '4'] 	


['', '2', '2', ''] 	
['', '32', '128', '2'] 	
['8', '8', '32', '8'] 	
['4', '4', '8', '4'] 	


['', '2', '2', ''] 	
['', '32', '128', '2'] 	
['8', '8', '32', '8'] 	
['4', '4', '8', '4'] 	


['8', '2', '2', '2'] 	
['4', '32', '128', '8'] 	
['2', '8', '32', '4'] 	
['', '4', '8', ''] 	


['8', '4', '2', '2'] 	
['4', '32', '128', '8'] 	
['2', '8', '32', '4'] 	
['4', '8', '', ''] 	


['', '8', '4', '4'] 	
['4', '32', '128

['2', '16', '2', '4'] 	
['256', '4', '32', '8'] 	
['2', '8', '64', '4'] 	
['', '16', '4', '2'] 	


['2', '16', '2', '4'] 	
['256', '4', '32', '8'] 	
['2', '8', '64', '4'] 	
['', '16', '4', '2'] 	


['2', '16', '2', '4'] 	
['256', '4', '32', '8'] 	
['2', '8', '64', '4'] 	
['16', '4', '2', '2'] 	


['2', '16', '2', '4'] 	
['256', '4', '32', '8'] 	
['2', '8', '64', '4'] 	
['4', '16', '4', '4'] 	


Episode:9 Score:132
[2, '', '', ''] 	
['', 4, '', ''] 	
['', '', '', ''] 	
['', '', '', ''] 	


['', '', '', '2'] 	
['', '', '', '4'] 	
['', '2', '', ''] 	
['', '', '', ''] 	


['2', '2', '', '2'] 	
['', '', '', '4'] 	
['', '', '', ''] 	
['', '', '', ''] 	


['', '', '', ''] 	
['', '', '', ''] 	
['', '', '', '2'] 	
['2', '2', '2', '4'] 	


['', '', '', ''] 	
['', '', '', ''] 	
['', '', '2', '2'] 	
['', '2', '4', '4'] 	


['', '2', '2', '2'] 	
['', '', '4', '4'] 	
['4', '', '', ''] 	
['', '', '', ''] 	


['4', '2', '', ''] 	
['8', '', '4', ''] 	
['4', '', '', ''] 	
['', '', '', ''] 	


['', '', '

# 2. Create a Deep Learning Model with Keras

In [11]:
import numpy as np
import tensorflow
# from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [12]:
states = env.observation_space.shape[0]
actions = env.action_space.n

In [13]:
print(states, type(states), actions, type(actions))

16 <class 'int'> 4 <class 'int'>


In [14]:
def build_model(states, actions):
    model = tensorflow.keras.Sequential()    
    model.add(Dense(128, activation='relu', input_shape=(1,states)))
#     model.add(Dropout(0.1)) not needed tbh, not limited dataset
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Flatten())
    model.add(Dense(actions, activation='linear'))
    return model

In [15]:
if 'model' in globals():
    del model 
    print('deleted model')

In [16]:
model = build_model(states, actions)

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1, 128)            2176      
_________________________________________________________________
dense_1 (Dense)              (None, 1, 64)             8256      
_________________________________________________________________
dense_2 (Dense)              (None, 1, 64)             4160      
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 260       
Total params: 14,852
Trainable params: 14,852
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [18]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [19]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [20]:
# tensorflow.compat.v1.logging.set_verbosity(tensorflow.compat.v1.logging.ERROR) # hide error / deprecated warning messages

dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=100_000, visualize=False, verbose=1)

Training for 100000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 7:31 - reward: 0.0000e+00

C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
C:\Users\user\anaconda3\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
C:\Users\user\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   51/10000 [..............................] - ETA: 1:50 - reward: -0.1373

C:\Users\user\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\user\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\user\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\user\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\user\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 15 + 1) inste

10000/10000 [==============================] - 53s 5ms/step - reward: -0.8492
11 episodes - episode_reward: -370.727 [-2114.000, 29.000] - loss: 53.611 - mae: 89.351 - mean_q: 129.170

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 53s 5ms/step - reward: -0.9211
5 episodes - episode_reward: -2728.200 [-13860.000, 65.000] - loss: 658.861 - mae: 234.641 - mean_q: 324.290

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 54s 5ms/step - reward: 0.5216
91 episodes - episode_reward: 57.220 [16.000, 116.000] - loss: 63999.328 - mae: 1230.274 - mean_q: 1672.963

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 56s 6ms/step - reward: 0.5133
88 episodes - episode_reward: 57.966 [17.000, 147.000] - loss: 397723.469 - mae: 4115.909 - mean_q: 5596.787

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 57s 6ms/step - reward: 0.5309
88 episodes - episode_reward: 60.648 

In [36]:
dqn.fit(env, nb_steps=200_000, visualize=False, verbose=1)

Training for 200000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 57s 6ms/step - reward: 0.5281
95 episodes - episode_reward: 55.105 [3.000, 124.000] - loss: 176615875539.048 - mae: 4298989.296 - mean_q: 5748376.281

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 57s 6ms/step - reward: 0.5266
96 episodes - episode_reward: 55.281 [17.000, 132.000] - loss: 234291625984.000 - mae: 5071117.500 - mean_q: 6771296.000

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 57s 6ms/step - reward: 0.5209
95 episodes - episode_reward: 54.832 [10.000, 145.000] - loss: 225405976576.000 - mae: 5060346.500 - mean_q: 6753852.000

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 57s 6ms/step - reward: 0.5123: 0s -
88 episodes - episode_reward: 57.182 [11.000, 123.000] - loss: 202462543872.000 - mae: 4934104.500 - mean_q: 6589625.000

Interval 5 (40000 steps performed

In [21]:
# scores = dqn.test(env, nb_episodes=10, visualize=False)
# print(np.mean(scores.history['episode_reward']))

In [22]:
# dqn.test(env, nb_episodes=1, visualize=True)

Help on DQNAgent in module rl.agents.dqn object:

class DQNAgent(AbstractDQNAgent)
 |  DQNAgent(model, policy=None, test_policy=None, enable_double_dqn=False, enable_dueling_network=False, dueling_type='avg', *args, **kwargs)
 |  
 |  # Arguments
 |      model__: A Keras model.
 |      policy__: A Keras-rl policy that are defined in [policy](https://github.com/keras-rl/keras-rl/blob/master/rl/policy.py).
 |      test_policy__: A Keras-rl policy.
 |      enable_double_dqn__: A boolean which enable target network as a second network proposed by van Hasselt et al. to decrease overfitting.
 |      enable_dueling_dqn__: A boolean which enable dueling architecture proposed by Mnih et al.
 |      dueling_type__: If `enable_dueling_dqn` is set to `True`, a type of dueling architecture must be chosen which calculate Q(s,a) from V(s) and A(s,a) differently. Note that `avg` is recommanded in the [paper](https://arxiv.org/abs/1511.06581).
 |          `avg`: Q(s,a;theta) = V(s;theta) + (A(s,a;theta

In [37]:
newgame = GameEnv()

count = 0
total_reward = 0

for i in range(200):
    move = dqn.forward(newgame.game.getboard())
    _, reward, done, _ = newgame.step(move)
    print('moving', move)
    newgame.game.printboard()
    print('')
    
    count += 1
    total_reward += reward
    
    if done:
        break
print('count:', count, '\n total rewards:', total_reward)
    

['', '', '', ''] 	
['', '', 4, ''] 	
['', '', '', ''] 	
['', '', 2, ''] 	
moving 1
['', '', '', '4'] 	
['', '', '', ''] 	
['', '', '4', ''] 	
['', '', '2', ''] 	

moving 0
['', '2', '4', '4'] 	
['', '', '2', ''] 	
['', '', '', ''] 	
['', '', '', ''] 	

moving 1
['', '', '', ''] 	
['', '', '', ''] 	
['', '', '4', '4'] 	
['', '2', '2', '4'] 	

moving 0
['', '2', '4', '8'] 	
['', '', '2', ''] 	
['', '', '', ''] 	
['', '', '', '2'] 	

moving 0
['', '2', '4', '8'] 	
['', '', '2', '2'] 	
['', '', '', '2'] 	
['', '', '', ''] 	

moving 0
['2', '2', '4', '8'] 	
['', '', '2', '4'] 	
['', '', '', ''] 	
['', '', '', ''] 	

moving 2
['4', '4', '8', ''] 	
['2', '4', '', ''] 	
['', '', '', ''] 	
['', '', '2', ''] 	

moving 1
['', '', '', ''] 	
['', '2', '', ''] 	
['4', '', '8', ''] 	
['2', '8', '2', ''] 	

moving 0
['4', '2', '8', ''] 	
['2', '8', '2', '2'] 	
['', '', '', ''] 	
['', '', '', ''] 	

moving 2
['4', '2', '8', '2'] 	
['2', '8', '4', ''] 	
['', '', '', ''] 	
['', '', '', ''] 	

moving 2
['

# 4. Reloading Agent from Memory

In [33]:
dqn.save_weights('dqn_weights_1286464_30k_.h5f', overwrite=True)

In [26]:
# del model
# del dqn
# del env

In [27]:
import gym
import numpy as np
import tensorflow
# from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [28]:
# env = gym.make('CartPole-v0')
# actions = env.action_space.n
# states = env.observation_space.shape[0]
# model = build_model(states, actions)
# dqn = build_agent(model, actions)
# dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [29]:
# dqn.load_weights('dqn_weights.h5f')

In [30]:
# dqn.test(GameEnv(), nb_episodes=1, visualize=True)